In [1]:
import sys
print(sys.version)

3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


In [2]:
import requests, base64, time
OM2M_URL = "http://127.0.0.1:8282/~"
CSE_ID = "/mn-cse/"
CSE_NAME = "mn-name"
LOGIN="admin"
PSWD="admin"
OM2M_BASE = OM2M_URL+CSE_ID
auth_headers = {"X-M2M-ORIGIN":LOGIN+":"+PSWD}
# The other accepted value is application/xml
common_headers = {"Accept": "application/json"}

def create_AE(name):
    header_ae = {"Content-Type":"application/xml;ty=2"}
    name_ae = name
    body_ae = """
    <m2m:ae xmlns:m2m="http://www.onem2m.org/xml/protocols" rn="{0}">
        <api>semantic</api>
        <lbl>Type/semantic Category/temperature Location/home</lbl>
        <rr>false</rr>
    </m2m:ae>
    """.format(name_ae)
    response = requests.delete(OM2M_BASE+CSE_NAME+"/"+name_ae, headers={**auth_headers, **common_headers})
    print(response)
    response = requests.post(OM2M_BASE, data=body_ae, headers={**auth_headers, **common_headers, **header_ae})
    print(response)
def create_CNT(name_ae, name_cnt):
    header_cnt = {"Content-Type":"application/xml;ty=3"}
    body_cnt = """<m2m:cnt xmlns:m2m="http://www.onem2m.org/xml/protocols" rn="{0}"></m2m:cnt>""".format(name_cnt)
    print(body_cnt)
    response = requests.post(OM2M_BASE+CSE_NAME+"/"+name_ae, data=body_cnt, headers={**auth_headers, **common_headers, **header_cnt})
    print(response)

def create_CIN(name_ae, name_cnt, mode, cin):
    header_cin = {"Content-Type":"application/xml;ty=4"}
    body_cin = """<m2m:cin xmlns:m2m="http://www.onem2m.org/xml/protocols"><cnf>{0}</cnf><con>{1}</con></m2m:cin>""".format(mode, base64.b64encode(cin).decode("utf-8"))
    print(body_cin)
    response = requests.post(OM2M_BASE+CSE_NAME+"/"+name_ae+"/"+name_cnt, data=body_cin, headers={**auth_headers, **common_headers, **header_cin})
    print(response)

In [3]:
create_AE("sm_sensor_1")
time.sleep(2)
create_CNT("sm_sensor_1", "sm_DATA")

<Response [404]>
<Response [201]>
<m2m:cnt xmlns:m2m="http://www.onem2m.org/xml/protocols" rn="sm_DATA"></m2m:cnt>
<Response [201]>


In [4]:
# 新增觀測值 (read)
read1 = b"""
@prefix exq: <http://example.org/ns#> .
@prefix sosa: <http://www.w3.org/ns/sosa/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

exq:obs005 a sosa:Observation ;
    sosa:hasResult 20 ;
    sosa:madeBySensor exq:thermometer ;
    sosa:observedProperty exq:Temperature ;
    sosa:resultTime "2020-06-20T11:30:12+00:00"^^xsd:dateTimeStamp .

exq:obs006 a sosa:Observation ;
    sosa:hasResult 32 ;
    sosa:madeBySensor exq:thermometer ;
    sosa:observedProperty exq:Temperature ;
    sosa:resultTime "2020-06-20T23:30:12+00:00"^^xsd:dateTimeStamp .
"""
create_CIN("sm_sensor_1", "sm_DATA", "read", read1)

<m2m:cin xmlns:m2m="http://www.onem2m.org/xml/protocols"><cnf>read</cnf><con>CkBwcmVmaXggZXhxOiA8aHR0cDovL2V4YW1wbGUub3JnL25zIz4gLgpAcHJlZml4IHNvc2E6IDxodHRwOi8vd3d3LnczLm9yZy9ucy9zb3NhLz4gLgpAcHJlZml4IHhzZDogPGh0dHA6Ly93d3cudzMub3JnLzIwMDEvWE1MU2NoZW1hIz4gLgoKZXhxOm9iczAwNSBhIHNvc2E6T2JzZXJ2YXRpb24gOwogICAgc29zYTpoYXNSZXN1bHQgMjAgOwogICAgc29zYTptYWRlQnlTZW5zb3IgZXhxOnRoZXJtb21ldGVyIDsKICAgIHNvc2E6b2JzZXJ2ZWRQcm9wZXJ0eSBleHE6VGVtcGVyYXR1cmUgOwogICAgc29zYTpyZXN1bHRUaW1lICIyMDIwLTA2LTIwVDExOjMwOjEyKzAwOjAwIl5eeHNkOmRhdGVUaW1lU3RhbXAgLgoKZXhxOm9iczAwNiBhIHNvc2E6T2JzZXJ2YXRpb24gOwogICAgc29zYTpoYXNSZXN1bHQgMzIgOwogICAgc29zYTptYWRlQnlTZW5zb3IgZXhxOnRoZXJtb21ldGVyIDsKICAgIHNvc2E6b2JzZXJ2ZWRQcm9wZXJ0eSBleHE6VGVtcGVyYXR1cmUgOwogICAgc29zYTpyZXN1bHRUaW1lICIyMDIwLTA2LTIwVDIzOjMwOjEyKzAwOjAwIl5eeHNkOmRhdGVUaW1lU3RhbXAgLgo=</con></m2m:cin>
<Response [201]>


In [5]:
# query
query1 = b"""
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX exq: <http://example.org/ns#>
SELECT ?result
WHERE {
    exq:obs006 a sosa:Observation;
        sosa:hasResult ?val
    BIND (
      COALESCE(
        IF(?val >= 30, "http://127.0.0.1:8080/~/mn-cse/mn-name/LAMP_1?op=setOn&lampid=LAMP_1", 1/0),
        IF(?val <= 25, "http://127.0.0.1:8080/~/mn-cse/mn-name/LAMP_1?op=setOff&lampid=LAMP_1", 1/0),
        "ERROR"
      ) AS ?result
    )
}
"""
create_CIN("sm_sensor_1", "sm_DATA", "query", query1)

<m2m:cin xmlns:m2m="http://www.onem2m.org/xml/protocols"><cnf>query</cnf><con>ClBSRUZJWCBzb3NhOiA8aHR0cDovL3d3dy53My5vcmcvbnMvc29zYS8+ClBSRUZJWCBleHE6IDxodHRwOi8vZXhhbXBsZS5vcmcvbnMjPgpTRUxFQ1QgP3Jlc3VsdApXSEVSRSB7CiAgICBleHE6b2JzMDA2IGEgc29zYTpPYnNlcnZhdGlvbjsKICAgICAgICBzb3NhOmhhc1Jlc3VsdCA/dmFsCiAgICBCSU5EICgKICAgICAgQ09BTEVTQ0UoCiAgICAgICAgSUYoP3ZhbCA+PSAzMCwgImh0dHA6Ly8xMjcuMC4wLjE6ODA4MC9+L21uLWNzZS9tbi1uYW1lL0xBTVBfMT9vcD1zZXRPbiZsYW1waWQ9TEFNUF8xIiwgMS8wKSwKICAgICAgICBJRig/dmFsIDw9IDI1LCAiaHR0cDovLzEyNy4wLjAuMTo4MDgwL34vbW4tY3NlL21uLW5hbWUvTEFNUF8xP29wPXNldE9mZiZsYW1waWQ9TEFNUF8xIiwgMS8wKSwKICAgICAgICAiRVJST1IiCiAgICAgICkgQVMgP3Jlc3VsdAogICAgKQp9Cg==</con></m2m:cin>
<Response [201]>


In [6]:
# 更新 (刪除) exq:obs006 觀測值
update2 = b"""
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX exq: <http://example.org/ns#>

DELETE { exq:obs006 ?property ?value } 
WHERE { exq:obs006 ?property ?value } 
"""
create_CIN("sm_sensor_1", "sm_DATA", "update", update2)


<m2m:cin xmlns:m2m="http://www.onem2m.org/xml/protocols"><cnf>update</cnf><con>ClBSRUZJWCBzb3NhOiA8aHR0cDovL3d3dy53My5vcmcvbnMvc29zYS8+ClBSRUZJWCBleHE6IDxodHRwOi8vZXhhbXBsZS5vcmcvbnMjPgoKREVMRVRFIHsgZXhxOm9iczAwNiA/cHJvcGVydHkgP3ZhbHVlIH0gCldIRVJFIHsgZXhxOm9iczAwNiA/cHJvcGVydHkgP3ZhbHVlIH0gCg==</con></m2m:cin>
<Response [201]>


In [7]:
# 新增 exq:obs006 觀測值
read1 = b"""
@prefix exq: <http://example.org/ns#> .
@prefix sosa: <http://www.w3.org/ns/sosa/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

exq:obs006 a sosa:Observation ;
    sosa:hasResult 23 ;
    sosa:madeBySensor exq:thermometer ;
    sosa:observedProperty exq:Temperature ;
    sosa:resultTime "2020-06-20T23:30:12+00:00"^^xsd:dateTimeStamp .
"""
create_CIN("sm_sensor_1", "sm_DATA", "read", read1)

<m2m:cin xmlns:m2m="http://www.onem2m.org/xml/protocols"><cnf>read</cnf><con>CkBwcmVmaXggZXhxOiA8aHR0cDovL2V4YW1wbGUub3JnL25zIz4gLgpAcHJlZml4IHNvc2E6IDxodHRwOi8vd3d3LnczLm9yZy9ucy9zb3NhLz4gLgpAcHJlZml4IHhzZDogPGh0dHA6Ly93d3cudzMub3JnLzIwMDEvWE1MU2NoZW1hIz4gLgoKZXhxOm9iczAwNiBhIHNvc2E6T2JzZXJ2YXRpb24gOwogICAgc29zYTpoYXNSZXN1bHQgMjMgOwogICAgc29zYTptYWRlQnlTZW5zb3IgZXhxOnRoZXJtb21ldGVyIDsKICAgIHNvc2E6b2JzZXJ2ZWRQcm9wZXJ0eSBleHE6VGVtcGVyYXR1cmUgOwogICAgc29zYTpyZXN1bHRUaW1lICIyMDIwLTA2LTIwVDIzOjMwOjEyKzAwOjAwIl5eeHNkOmRhdGVUaW1lU3RhbXAgLgo=</con></m2m:cin>
<Response [201]>


In [9]:
# query
query1 = b"""
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX exq: <http://example.org/ns#>
SELECT ?result
WHERE {
    exq:obs006 a sosa:Observation;
        sosa:hasResult ?val
    BIND (
      COALESCE(
        IF(?val >= 30, "http://127.0.0.1:8080/~/mn-cse/mn-name/LAMP_1?op=setOn&lampid=LAMP_1", 1/0),
        IF(?val <= 25, "http://127.0.0.1:8080/~/mn-cse/mn-name/LAMP_1?op=setOff&lampid=LAMP_1", 1/0),
        "ERROR"
      ) AS ?result
    )
}
"""
create_CIN("sm_sensor_1", "sm_DATA", "query", query1)


<m2m:cin xmlns:m2m="http://www.onem2m.org/xml/protocols"><cnf>query</cnf><con>ClBSRUZJWCBzb3NhOiA8aHR0cDovL3d3dy53My5vcmcvbnMvc29zYS8+ClBSRUZJWCBleHE6IDxodHRwOi8vZXhhbXBsZS5vcmcvbnMjPgpTRUxFQ1QgP3Jlc3VsdApXSEVSRSB7CiAgICBleHE6b2JzMDA2IGEgc29zYTpPYnNlcnZhdGlvbjsKICAgICAgICBzb3NhOmhhc1Jlc3VsdCA/dmFsCiAgICBCSU5EICgKICAgICAgQ09BTEVTQ0UoCiAgICAgICAgSUYoP3ZhbCA+PSAzMCwgImh0dHA6Ly8xMjcuMC4wLjE6ODA4MC9+L21uLWNzZS9tbi1uYW1lL0xBTVBfMT9vcD1zZXRPbiZsYW1waWQ9TEFNUF8xIiwgMS8wKSwKICAgICAgICBJRig/dmFsIDw9IDI1LCAiaHR0cDovLzEyNy4wLjAuMTo4MDgwL34vbW4tY3NlL21uLW5hbWUvTEFNUF8xP29wPXNldE9mZiZsYW1waWQ9TEFNUF8xIiwgMS8wKSwKICAgICAgICAiRVJST1IiCiAgICAgICkgQVMgP3Jlc3VsdAogICAgKQp9Cg==</con></m2m:cin>
<Response [201]>
